<a href="https://colab.research.google.com/github/Sabelz/Master_Thesis_Alexander/blob/main/utils/models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The basic structure of all Gaussian process models

# Imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Master_Thesis_Alexander/
!git config --global user.email "alexander.sabelstrom.1040@student.uu.se"
!git config --global user.name "Sabelz"

import numpy as np
import matplotlib.pyplot as plt
import torch
!pip install gpytorch > \dev\null # Suppress prints, remove "> \dev\null" for potential error messages
import gpytorch
# Define the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("----------------------------------------------------------")
print("ALL MODELS: ")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Master_Thesis_Alexander
----------------------------------------------------------
ALL MODELS: 


# Structured Kernel Interpollation (SKI/KISS-GP)   
https://docs.gpytorch.ai/en/latest/examples/02_Scalable_Exact_GPs/KISSGP_Regression.html

## 1D-4D data

In [ ]:
class KISSGP(gpytorch.models.ExactGP):
    def __init__(self, x_train, y_train, likelihood, mean, kernel):
        super(KISSGP, self).__init__(x_train, y_train, likelihood)

        # SKI requires a grid size hyperparameter. This util can help with that.
        grid_size = gpytorch.utils.grid.choose_grid_size(x_train)
        self.mean_module = mean
        # Check dimension of x_train
        if(x_train.dim() > 1):
          dim = x_train.shape[1]
        else:
          dim = 1
        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.GridInterpolationKernel(
                kernel, grid_size=grid_size, num_dims = dim # Get dimension of x
            )
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)
print("""KISS-GP For 1D-4D data:
    Example:
      likelihood = gpytorch.likelihoods.GaussianLikelihood()
      mean = gpytorch.means.ConstantMean()
      kernel = gpytorch.kernels.RBFKernel()
      model = KISSGP(x_train, y_train, likelihood, mean, kernel)
      model = model.to(device) # Move model to device""")

KISS-GP For 1D-4D data:
    Example:
      likelihood = gpytorch.likelihoods.GaussianLikelihood()
      mean = gpytorch.means.ConstantMean()
      kernel = gpytorch.kernels.RBFKernel()
      model = KISSGP(x_train, y_train, likelihood, mean, kernel)
      model = model.to(device) # Move model to device


## For higher dimensional data

In [ ]:
class KISSGP_NDim(gpytorch.models.ExactGP):
    def __init__(self, x_train, y_train, likelihood, mean, kernel):
        super(KISSGP_NDim, self).__init__(x_train, y_train, likelihood)

        # SKI requires a grid size hyperparameter. This util can help with that
        # We're setting Kronecker structure to False because we're using an additive structure decomposition
        grid_size = gpytorch.utils.grid.choose_grid_size(x_train, kronecker_structure=False)

        self.mean_module = mean
        self.covar_module = gpytorch.kernels.AdditiveStructureKernel(
            gpytorch.kernels.ScaleKernel(
                gpytorch.kernels.GridInterpolationKernel(
                    kernel, grid_size=128, num_dims=1
                )
            ), num_dims=x_train.shape[1]# Get dimension of training data
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

print("")
print("""KISS-GP For higher dimensional data:
    Example:
      likelihood = gpytorch.likelihoods.GaussianLikelihood()
      mean = gpytorch.means.ConstantMean()
      kernel = gpytorch.kernels.RBFKernel()
      model = KISSGP_NDim(x_train, y_train, likelihood, mean, kernel)
      model = model.to(device) # Move model to device""")



KISS-GP For higher dimensional data:
    Example:
      likelihood = gpytorch.likelihoods.GaussianLikelihood()
      mean = gpytorch.means.ConstantMean()
      kernel = gpytorch.kernels.RBFKernel()
      model = KISSGP_NDim(x_train, y_train, likelihood, mean, kernel)
      model = model.to(device) # Move model to device


# Variational Distribution

In [ ]:
class variationalGP(gpytorch.models.ApproximateGP):
    def __init__(self, likelihood, mean, kernel, inducing_points):
        variational_distribution = gpytorch.variational.CholeskyVariationalDistribution(inducing_points.size(0))
        variational_strategy = gpytorch.variational.VariationalStrategy(
            self, inducing_points, variational_distribution, learn_inducing_locations=True)
        super().__init__(variational_strategy)
        self.mean_module = mean
        self.covar_module = gpytorch.kernels.ScaleKernel(kernel)

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)


print("")
print(""" Variational GP:
    Example:
      likelihood = gpytorch.likelihoods.GaussianLikelihood()
      mean = gpytorch.means.ConstantMean()
      kernel = gpytorch.kernels.RBFKernel()
      n_inducing_points = 500
      inducing_points = x_train[torch.randperm(x_train.size(0))[:num_inducing_points]]
      model = variationalGP(likelihood, mean, kernel, inducing_points)
      model = model.to(device) # Move model to device.""")



Inducing Points GP:
    Example:
      likelihood = gpytorch.likelihoods.GaussianLikelihood()
      mean = gpytorch.means.ConstantMean()
      kernel = gpytorch.kernels.RBFKernel()
      n_inducing_points = 500
      inducing_points = x_train[torch.randperm(x_train.size(0))[:num_inducing_points]]
      model = InducingGP(likelihood, mean, kernel, inducing_points)
      model = model.to(device) # Move model to device.


# State Space Model